## 0. Introduction
Objectif: comparer plusieurs modèles de classification (baseline, LogReg, RF, XGB).

Traçabilité et reproductibilité via MLflow.


## 1. Import & configuration

In [23]:
import sys, os
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import (
    RocCurveDisplay,
    PrecisionRecallDisplay,
)
import seaborn as sns
from matplotlib import pyplot as plt
from xgboost import XGBClassifier
import pandas as pd
from rich import print

# Dataset + constantes
sys.path.insert(0, "..")
from config import PROCESSED_DATA_PATH, TARGET_COL, SEED, FIG_DIR, GLOBAL_SCORING, CV_STRATEGY
from utils import compute_final_metrics, compute_overfitting_metrics, compute_train_test_metrics, extract_cv_metrics

df = pd.read_csv(PROCESSED_DATA_PATH)
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# --- Identifier les colonnes numériques et binaires (déterminées par feature engineering)
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
# Retirer les colonnes binaires (valeurs 0/1 ou très peu de valeurs uniques)
binary_cols = [col for col in numeric_cols if X[col].nunique() == 2 or (X[col].dtype == 'int64' and set(X[col].unique()) == {0, 1})]
numeric_cols = [col for col in numeric_cols if col not in binary_cols]

NUMERIC_COLS = numeric_cols
BINARY_COLS = binary_cols

print(f"[bold cyan]📊 Colonnes détectées par feature engineering:[/bold cyan]")
print(f"   - Numériques: {NUMERIC_COLS}")
print(f"   - Binaires: {BINARY_COLS}")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "false"
mlflow.autolog(disable=True)  # évite les runs fantômes
mlflow.end_run()  # ferme tout run résiduel
mlflow.set_tracking_uri("file:../mlruns")
mlflow.set_experiment("Rugby Kicks Classification - Benchmark")
mlflow.set_tag("author", "Xavier")
mlflow.set_tag("project", "OC P6")

X_trainfull = pd.concat([X_train, y_train], axis=1)
X_testfull = pd.concat([X_test, y_test], axis=1)

train_dataset = mlflow.data.from_pandas(X_trainfull, source=PROCESSED_DATA_PATH, name="Training_set")
test_dataset = mlflow.data.from_pandas(X_testfull, source=PROCESSED_DATA_PATH, name="Test_set")

if mlflow.active_run():
    print("⚠️ Run actif trouvé :", mlflow.active_run().info.run_id)
    mlflow.end_run()

print("[bold green]✅ Configuration MLflow prête[/bold green]")

📊 Colonnes détectées par feature engineering:

- Numériques: ['time_norm', 'distance', 'angle', 'wind_speed', 'precipitation_probability']

- Binaires: ['is_left_footed', 'game_away', 'is_endgame', 'is_start', 'is_left_side', 'has_previous_attempts']

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/data/processed/kicks_ready_for_model.csv'. Exception: 
  return _dataset_source_registry.resolve(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '/Users/xaviercou

⚠️ Run actif trouvé : db2bf5df6b1c4ec19ecff339506b9b37

✅ Configuration MLflow prête

In [24]:
# ============================================================
# 🔧 PREPROCESSING & FEATURE SCALING (Pipeline)
# ============================================================
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Créer la pipeline de preprocessing
# - StandardScaler pour colonnes numériques
# - Passthrough pour colonnes binaires (déjà 0/1)
preprocessing_pipeline = ColumnTransformer([
    ('scaler', StandardScaler(), NUMERIC_COLS),
    ('passthrough', 'passthrough', BINARY_COLS)
], remainder='drop')

# FIT sur X_train UNIQUEMENT (prevent data leakage!)
X_train_processed = preprocessing_pipeline.fit_transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)

# Convertir en DataFrame pour maintenabilité
feature_names = NUMERIC_COLS + BINARY_COLS
X_train_processed = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_processed = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

print("[bold green]✅ Pipeline de preprocessing créée et appliquée[/bold green]")
print(f"   - Colonnes numériques standardisées: {NUMERIC_COLS}")
print(f"   - Colonnes binaires conservées: {BINARY_COLS}")
print(f"   - X_train shape: {X_train_processed.shape}")
print(f"   - X_test shape: {X_test_processed.shape}")

✅ Pipeline de preprocessing créée et appliquée

- Colonnes numériques standardisées: ['time_norm', 'distance', 'angle', 'wind_speed', 
'precipitation_probability']

- Colonnes binaires conservées: ['is_left_footed', 'game_away', 'is_endgame', 'is_start', 'is_left_side', 
'has_previous_attempts']

- X_train shape: (13873, 11)

- X_test shape: (3469, 11)

## 2. Baseline Dummy Classifier

In [25]:
with mlflow.start_run(
    run_name="Dummy_Baseline", description="Baseline avec stratégie 'most_frequent'"
):

    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(test_dataset, context="testing")
    mlflow.set_tag("model_type", "dummy")

    # --- Définir les hyperparamètres (AVANT la création du modèle)
    model_params = {
        "strategy": "most_frequent",
        "random_state": SEED,
    }
    
    # --- Logger les paramètres EN PREMIER
    mlflow.log_params(model_params)

    # --- Créer le modèle avec les params
    model = DummyClassifier(**model_params)

    # --- Cross-Validation (avec données preprocessées)
    cv_results = cross_validate(
        model,
        X_train_processed,
        y_train,
        cv=CV_STRATEGY,
        scoring=GLOBAL_SCORING,
        return_train_score=True,
    )

    cv_mean, cv_std, train_mean = extract_cv_metrics(cv_results)

    # --- Entraînement final (avec données preprocessées)
    model.fit(X_train_processed, y_train)

    # --- Prédictions finales (avec données preprocessées)
    y_pred_train = model.predict(X_train_processed)
    y_proba_train = np.asarray(model.predict_proba(X_train_processed))[:, 1]
    
    y_pred = model.predict(X_test_processed)
    y_proba = np.asarray(model.predict_proba(X_test_processed))[:, 1]

    # --- Métriques train/test + overfitting gaps
    final_metrics_test, cm, (fp_rate, fn_rate) = compute_train_test_metrics(
        y_train, y_pred_train, y_proba_train,
        y_test, y_pred, y_proba
    )
    
    # --- Log metrics
    mlflow.log_metrics(cv_mean)
    mlflow.log_metrics(cv_std)
    mlflow.log_metrics(train_mean)
    mlflow.log_metrics(final_metrics_test)

    mlflow.sklearn.log_model(model, name="dummy_model", input_example=X_test_processed.iloc[:5])

    # --- Matrice de confusion
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title("Confusion Matrix - Dummy Baseline")
    plt.xlabel("Prédictions")
    plt.ylabel("Vérités terrain")
    plt.tight_layout()
    cm_path = f"{FIG_DIR}/dummy_confusion_matrix.png"
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)
    plt.close()

    # --- Courbe ROC
    RocCurveDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("ROC Curve - Dummy Baseline")
    plt.tight_layout()
    roc_path = f"{FIG_DIR}/dummy_roc_curve.png"
    plt.savefig(roc_path)
    mlflow.log_artifact(roc_path)
    plt.close()

    # --- Courbe Precision-Recall
    PrecisionRecallDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("Precision-Recall Curve - Dummy Baseline")
    plt.tight_layout()
    pr_path = f"{FIG_DIR}/dummy_pr_curve.png"
    plt.savefig(pr_path)
    mlflow.log_artifact(pr_path)
    plt.close()

    print(f"[bold cyan]✅ DummyClassifier logué dans MLflow avec succès[/bold cyan]")

# 🔎 Résumé local
print(
    f"""
📊 Résumé des métriques - Dummy Baseline :
  Train Accuracy: {final_metrics_test['train_accuracy']:.3f}
  Test Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  Overfitting Gap (Accuracy): {final_metrics_test['overfit_gap_accuracy']:.3f}
  ---
  Train F1 classe 0: {final_metrics_test['train_f1_0']:.3f}
  Test F1 classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Overfitting Gap (F1_0): {final_metrics_test['overfit_gap_f1_0']:.3f}
  ---
  Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  ROC-AUC:   {final_metrics_test['final_test_auc']:.3f}
  Precision classe 0: {final_metrics_test['final_test_precision_0']:.3f}
  Recall classe 0:    {final_metrics_test['final_test_recall_0']:.3f}
  F1-score classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Precision classe 1: {final_metrics_test['final_test_precision_1']:.3f}
  Recall classe 1:    {final_metrics_test['final_test_recall_1']:.3f}
  F1-score classe 1:  {final_metrics_test['final_test_f1_1']:.3f}
  FPR:       {fp_rate:.3f}
  FNR:       {fn_rate:.3f}
"""
)

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python 

✅ DummyClassifier logué dans MLflow avec succès

📊 Résumé des métriques - Dummy Baseline :
  Train Accuracy: 0.764
  Test Accuracy:  0.764
  Overfitting Gap (Accuracy): 0.000
  ---
  Train F1 classe 0: 0.000
  Test F1 classe 0:  0.000
  Overfitting Gap (F1_0): 0.000
  ---
  Accuracy:  0.764
  ROC-AUC:   0.500
  Precision classe 0: 0.000
  Recall classe 0:    0.000
  F1-score classe 0:  0.000
  Precision classe 1: 0.764
  Recall classe 1:    1.000
  F1-score classe 1:  0.866
  FPR:       1.000
  FNR:       0.000

## 3. Logistic Regression Classifier

In [26]:
import mlflow.sklearn as mlflow_sklearn

with mlflow.start_run(
    run_name="LogisticRegression_v1",
    description="Régression logistique binaire – baseline explicative",
):
    
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(test_dataset, context="testing")
    mlflow.set_tag("model_type", "logistic_regression")

    # --- Définir les hyperparamètres (AVANT la création du modèle)
    model_params = {
        "solver": "liblinear",
        "penalty": "l2",
        "C": 1.0,
        "random_state": SEED,
        "class_weight": "balanced",
    }
    
    # --- Logger les paramètres EN PREMIER
    mlflow.log_params(model_params)

    # --- Créer le modèle avec les params
    model = LogisticRegression(**model_params)

    # --- Cross-Validation (avec données preprocessées)
    cv_results = cross_validate(
        model,
        X_train_processed,
        y_train,
        cv=CV_STRATEGY,
        scoring=GLOBAL_SCORING,
        return_train_score=True,
    )
    
    cv_mean, cv_std, train_mean = extract_cv_metrics(cv_results)

    # --- Entraînement final
    model.fit(X_train_processed, y_train)

    # --- Prédictions finales
    y_pred_train = model.predict(X_train_processed)
    y_proba_train = model.predict_proba(X_train_processed)[:, 1]
    
    y_pred = model.predict(X_test_processed)
    y_proba = model.predict_proba(X_test_processed)[:, 1]

    # --- Métriques train/test + overfitting gaps
    final_metrics_test, cm, (fp_rate, fn_rate) = compute_train_test_metrics(
        y_train, y_pred_train, y_proba_train,
        y_test, y_pred, y_proba
    )

    # --- Log metrics
    mlflow.log_metrics(cv_mean)
    mlflow.log_metrics(cv_std)
    mlflow.log_metrics(train_mean)
    mlflow.log_metrics(final_metrics_test)

    # --- Log du modèle
    mlflow_sklearn.log_model(
        sk_model=model,
        artifact_path="logreg_model",
        input_example=X_test_processed.iloc[:5],
    )

    # --- Matrice de confusion
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
    plt.title("Confusion Matrix - Logistic Regression")
    plt.xlabel("Prédictions")
    plt.ylabel("Vérités terrain")
    plt.tight_layout()
    cm_path = f"{FIG_DIR}/logreg_confusion_matrix.png"
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)
    plt.close()

    # --- Courbe ROC
    RocCurveDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("ROC Curve - Logistic Regression")
    plt.tight_layout()
    roc_path = f"{FIG_DIR}/logreg_roc_curve.png"
    plt.savefig(roc_path)
    mlflow.log_artifact(roc_path)
    plt.close()

    # --- Courbe Precision-Recall
    PrecisionRecallDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("Precision-Recall Curve - Logistic Regression")
    plt.tight_layout()
    pr_path = f"{FIG_DIR}/logreg_pr_curve.png"
    plt.savefig(pr_path)
    mlflow.log_artifact(pr_path)
    plt.close()

    print("[bold cyan]✅ LogisticRegression logué dans MLflow avec succès[/bold cyan]")

# 🔎 Résumé local
print(
    f"""
📊 Résumé des métriques - LogisticRegression :
  Train Accuracy: {final_metrics_test['train_accuracy']:.3f}
  Test Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  Overfitting Gap (Accuracy): {final_metrics_test['overfit_gap_accuracy']:.3f}
  ---
  Train F1 classe 0: {final_metrics_test['train_f1_0']:.3f}
  Test F1 classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Overfitting Gap (F1_0): {final_metrics_test['overfit_gap_f1_0']:.3f}
  ---
  Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  ROC-AUC:   {final_metrics_test['final_test_auc']:.3f}
  Precision classe 0: {final_metrics_test['final_test_precision_0']:.3f}
  Recall classe 0:    {final_metrics_test['final_test_recall_0']:.3f}
  F1-score classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Precision classe 1: {final_metrics_test['final_test_precision_1']:.3f}
  Recall classe 1:    {final_metrics_test['final_test_recall_1']:.3f}
  F1-score classe 1:  {final_metrics_test['final_test_f1_1']:.3f}
  FPR:       {fp_rate:.3f}
  FNR:       {fn_rate:.3f}
"""
)

2025/11/13 15:29:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/ty

✅ LogisticRegression logué dans MLflow avec succès

📊 Résumé des métriques - LogisticRegression :
  Train Accuracy: 0.680
  Test Accuracy:  0.697
  Overfitting Gap (Accuracy): -0.017
  ---
  Train F1 classe 0: 0.538
  Test F1 classe 0:  0.550
  Overfitting Gap (F1_0): -0.012
  ---
  Accuracy:  0.697
  ROC-AUC:   0.793
  Precision classe 0: 0.423
  Recall classe 0:    0.786
  F1-score classe 0:  0.550
  Precision classe 1: 0.910
  Recall classe 1:    0.670
  F1-score classe 1:  0.772
  FPR:       0.214
  FNR:       0.330

## 4. Random Forest Classifier

In [27]:
with mlflow.start_run(
    run_name="RandomForest_v1",
    description="Random Forest Classifier - modèle non linéaire",
):
    
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(test_dataset, context="testing")
    mlflow.set_tag("model_type", "random_forest")

    # --- Définir les hyperparamètres (AVANT la création du modèle)
    model_params = {
        "n_estimators": 200,
        "max_depth": None,
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "class_weight": "balanced",
        "random_state": SEED,
        "n_jobs": -1,
    }
    
    # --- Logger les paramètres EN PREMIER (convertir None en string pour MLflow)
    mlflow_params = {k: str(v) for k, v in model_params.items()}
    mlflow.log_params(mlflow_params)

    # --- Créer le modèle avec les params
    model = RandomForestClassifier(**model_params)

    # --- Cross-Validation
    cv_results = cross_validate(
        model,
        X_train_processed,
        y_train,
        cv=CV_STRATEGY,
        scoring=GLOBAL_SCORING,
        return_train_score=True,
    )
    
    cv_mean, cv_std, train_mean = extract_cv_metrics(cv_results)

    # --- Entraînement final
    model.fit(X_train_processed, y_train)

    # --- Prédictions finales
    y_pred_train = model.predict(X_train_processed)
    y_proba_train = model.predict_proba(X_train_processed)[:, 1]
    
    y_pred = model.predict(X_test_processed)
    y_proba = model.predict_proba(X_test_processed)[:, 1]

    # --- Métriques train/test + overfitting gaps
    final_metrics_test, cm, (fp_rate, fn_rate) = compute_train_test_metrics(
        y_train, y_pred_train, y_proba_train,
        y_test, y_pred, y_proba
    )
    
    # --- Log metrics
    mlflow.log_metrics(cv_mean)
    mlflow.log_metrics(cv_std)
    mlflow.log_metrics(train_mean)
    mlflow.log_metrics(final_metrics_test)

    # --- Log du modèle
    mlflow.sklearn.log_model(
        sk_model=model,
        name="random_forest_model",
        input_example=X_test_processed.iloc[:5],
    )

    # --- Matrice de confusion
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Purples", cbar=False)
    plt.title("Confusion Matrix - Random Forest")
    plt.xlabel("Prédictions")
    plt.ylabel("Vérités terrain")
    plt.tight_layout()
    cm_path = f"{FIG_DIR}/rf_confusion_matrix.png"
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)
    plt.close()

    # --- Courbe ROC
    RocCurveDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("ROC Curve - Random Forest")
    plt.tight_layout()
    roc_path = f"{FIG_DIR}/rf_roc_curve.png"
    plt.savefig(roc_path)
    mlflow.log_artifact(roc_path)
    plt.close()

    # --- Courbe Precision-Recall
    PrecisionRecallDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("Precision-Recall Curve - Random Forest")
    plt.tight_layout()
    pr_path = f"{FIG_DIR}/rf_pr_curve.png"
    plt.savefig(pr_path)
    mlflow.log_artifact(pr_path)
    plt.close()

    # --- Importance des features
    importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(
        ascending=False
    )
    plt.figure(figsize=(8, 5))
    imp_df = pd.DataFrame({'importance': importances[:10].values, 'feature': importances[:10].index})
    sns.barplot(data=imp_df, x='importance', y='feature', palette="viridis", hue='feature', legend=False)
    plt.title("Top 10 Feature Importances - Random Forest")
    plt.tight_layout()
    feat_imp_path = f"{FIG_DIR}/rf_feature_importances.png"
    plt.savefig(feat_imp_path)
    mlflow.log_artifact(feat_imp_path)
    plt.close()

    print("[bold cyan]✅ RandomForest logué dans MLflow avec succès[/bold cyan]")

# 🔎 Résumé local
print(
    f"""
📊 Résumé des métriques - RandomForest :
  Train Accuracy: {final_metrics_test['train_accuracy']:.3f}
  Test Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  Overfitting Gap (Accuracy): {final_metrics_test['overfit_gap_accuracy']:.3f}
  ---
  Train F1 classe 0: {final_metrics_test['train_f1_0']:.3f}
  Test F1 classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Overfitting Gap (F1_0): {final_metrics_test['overfit_gap_f1_0']:.3f}
  ---
  Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  ROC-AUC:   {final_metrics_test['final_test_auc']:.3f}
  Precision classe 0: {final_metrics_test['final_test_precision_0']:.3f}
  Recall classe 0:    {final_metrics_test['final_test_recall_0']:.3f}
  F1-score classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Precision classe 1: {final_metrics_test['final_test_precision_1']:.3f}
  Recall classe 1:    {final_metrics_test['final_test_recall_1']:.3f}
  F1-score classe 1:  {final_metrics_test['final_test_f1_1']:.3f}
  FPR:       {fp_rate:.3f}
  FNR:       {fn_rate:.3f}
"""
)

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/13 15:29:26 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/13 15:29:26 WARNI

✅ RandomForest logué dans MLflow avec succès

📊 Résumé des métriques - RandomForest :
  Train Accuracy: 1.000
  Test Accuracy:  0.756
  Overfitting Gap (Accuracy): 0.243
  ---
  Train F1 classe 0: 0.999
  Test F1 classe 0:  0.276
  Overfitting Gap (F1_0): 0.723
  ---
  Accuracy:  0.756
  ROC-AUC:   0.768
  Precision classe 0: 0.461
  Recall classe 0:    0.197
  F1-score classe 0:  0.276
  Precision classe 1: 0.789
  Recall classe 1:    0.929
  F1-score classe 1:  0.854
  FPR:       0.803
  FNR:       0.071

## 5. Support Vector Machine (SVM) Classifier

In [28]:
from sklearn.svm import SVC

with mlflow.start_run(
    run_name="SVM_v1",
    description="Support Vector Machine Classifier - noyau RBF avec class_weight",
):
    
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(test_dataset, context="testing")
    mlflow.set_tag("model_type", "svm")

    # --- Définir les hyperparamètres (AVANT la création du modèle)
    # Calcul du poids inversé pour gérer le déséquilibre de classe
    neg = (y_train == 0).sum()  # count of negative class (0)
    pos = (y_train == 1).sum()  # count of positive class (1)
    class_weights = {0: pos / neg, 1: 1.0}  # classe 0 (minoritaire) reçoit plus de poids
    
    model_params = {
        "kernel": "linear",  # Changed from "rbf" to "linear" for faster training
        "C": 0.5,  # Reduced from 1.0 for better generalization
        "gamma": "scale",
        "class_weight": class_weights,
        "probability": True,
        "random_state": SEED,
        "max_iter": 5000,  # Increased from 2000 for convergence (data is already scaled)
    }
    
    # --- Logger les paramètres EN PREMIER
    mlflow_params = model_params.copy()
    mlflow_params["class_weight"] = str(class_weights)  # convertir dict en string pour MLflow
    mlflow.log_params(mlflow_params)

    # --- Créer le modèle avec les params
    model = SVC(**model_params)

    # --- Cross-Validation
    cv_results = cross_validate(
        model,
        X_train_processed,
        y_train,
        cv=CV_STRATEGY,
        scoring=GLOBAL_SCORING,
        return_train_score=True,
    )
    
    cv_mean, cv_std, train_mean = extract_cv_metrics(cv_results)

    # --- Entraînement final
    model.fit(X_train_processed, y_train)

    # --- Prédictions finales
    y_pred_train = model.predict(X_train_processed)
    y_proba_train = model.predict_proba(X_train_processed)[:, 1]
    
    y_pred = model.predict(X_test_processed)
    y_proba = model.predict_proba(X_test_processed)[:, 1]

    # --- Métriques train/test + overfitting gaps
    final_metrics_test, cm, (fp_rate, fn_rate) = compute_train_test_metrics(
        y_train, y_pred_train, y_proba_train,
        y_test, y_pred, y_proba
    )
    
    # --- Log metrics
    mlflow.log_metrics(cv_mean)
    mlflow.log_metrics(cv_std)
    mlflow.log_metrics(train_mean)
    mlflow.log_metrics(final_metrics_test)

    # --- Log du modèle
    mlflow.sklearn.log_model(
        sk_model=model,
        name="svm_model",
        input_example=X_test_processed.iloc[:5],
    )

    # --- Matrice de confusion
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Oranges", cbar=False)
    plt.title("Confusion Matrix - SVM")
    plt.xlabel("Prédictions")
    plt.ylabel("Vérités terrain")
    plt.tight_layout()
    cm_path = f"{FIG_DIR}/svm_confusion_matrix.png"
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)
    plt.close()

    # --- Courbe ROC
    RocCurveDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("ROC Curve - SVM")
    plt.tight_layout()
    roc_path = f"{FIG_DIR}/svm_roc_curve.png"
    plt.savefig(roc_path)
    mlflow.log_artifact(roc_path)
    plt.close()

    # --- Courbe Precision-Recall
    PrecisionRecallDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("Precision-Recall Curve - SVM")
    plt.tight_layout()
    pr_path = f"{FIG_DIR}/svm_pr_curve.png"
    plt.savefig(pr_path)
    mlflow.log_artifact(pr_path)
    plt.close()

    print("[bold cyan]✅ SVM logué dans MLflow avec succès[/bold cyan]")

# 🔎 Résumé local
print(
    f"""
📊 Résumé des métriques - SVM :
  Train Accuracy: {final_metrics_test['train_accuracy']:.3f}
  Test Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  Overfitting Gap (Accuracy): {final_metrics_test['overfit_gap_accuracy']:.3f}
  ---
  Train F1 classe 0: {final_metrics_test['train_f1_0']:.3f}
  Test F1 classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Overfitting Gap (F1_0): {final_metrics_test['overfit_gap_f1_0']:.3f}
  ---
  Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  ROC-AUC:   {final_metrics_test['final_test_auc']:.3f}
  Precision classe 0: {final_metrics_test['final_test_precision_0']:.3f}
  Recall classe 0:    {final_metrics_test['final_test_recall_0']:.3f}
  F1-score classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Precision classe 1: {final_metrics_test['final_test_precision_1']:.3f}
  Recall classe 1:    {final_metrics_test['final_test_recall_1']:.3f}
  F1-score classe 1:  {final_metrics_test['final_test_f1_1']:.3f}
  FPR:       {fp_rate:.3f}
  FNR:       {fn_rate:.3f}
"""
)

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with S

✅ SVM logué dans MLflow avec succès

📊 Résumé des métriques - SVM :
  Train Accuracy: 0.568
  Test Accuracy:  0.572
  Overfitting Gap (Accuracy): -0.005
  ---
  Train F1 classe 0: 0.351
  Test F1 classe 0:  0.337
  Overfitting Gap (F1_0): 0.014
  ---
  Accuracy:  0.572
  ROC-AUC:   0.551
  Precision classe 0: 0.265
  Recall classe 0:    0.461
  F1-score classe 0:  0.337
  Precision classe 1: 0.785
  Recall classe 1:    0.607
  F1-score classe 1:  0.684
  FPR:       0.539
  FNR:       0.393

## 6. XGBoost Classifier

In [29]:
# Calcul du poids de la classe majoritaire / minoritaire
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

with mlflow.start_run(
    run_name="XGBoost_v1",
    description="XGBoost Classifier - modèle non linéaire avec scale_pos_weight",
):
    
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(test_dataset, context="testing")
    mlflow.set_tag("model_type", "xgboost")

    # --- Définir les hyperparamètres (AVANT la création du modèle)
    model_params = {
        "n_estimators": 300,
        "max_depth": 5,
        "min_child_weight": 3,
        "max_delta_step": 1,
        "learning_rate": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "scale_pos_weight": scale_pos_weight,
        "random_state": SEED,
        "eval_metric": "logloss",
        "n_jobs": -1,
    }
    
    # --- Logger les paramètres EN PREMIER (arrondir scale_pos_weight)
    mlflow_params = model_params.copy()
    mlflow_params["scale_pos_weight"] = round(scale_pos_weight, 2)
    mlflow.log_params(mlflow_params)

    # --- Créer le modèle avec les params
    model = XGBClassifier(**model_params)

    # --- Cross-Validation
    cv_results = cross_validate(
        model,
        X_train_processed,
        y_train,
        cv=CV_STRATEGY,
        scoring=GLOBAL_SCORING,
        return_train_score=True,
    )
    
    cv_mean, cv_std, train_mean = extract_cv_metrics(cv_results)

    # --- Entraînement final
    model.fit(X_train_processed, y_train)

    # --- Prédictions finales
    y_pred_train = model.predict(X_train_processed)
    y_proba_train = model.predict_proba(X_train_processed)[:, 1]
    
    y_pred = model.predict(X_test_processed)
    y_proba = model.predict_proba(X_test_processed)[:, 1]

    # --- Métriques train/test + overfitting gaps
    final_metrics_test, cm, (fp_rate, fn_rate) = compute_train_test_metrics(
        y_train, y_pred_train, y_proba_train,
        y_test, y_pred, y_proba
    )

    # --- Log metrics
    mlflow.log_metrics(cv_mean)
    mlflow.log_metrics(cv_std)
    mlflow.log_metrics(train_mean)
    mlflow.log_metrics(final_metrics_test)

    # --- Log modèle
    mlflow.xgboost.log_model(
        xgb_model=model,
        name="xgboost_model",
        input_example=X_test_processed.iloc[:5],
    )

    # --- Matrice de confusion
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="mako", cbar=False)
    plt.title("Confusion Matrix - XGBoost")
    plt.xlabel("Prédictions")
    plt.ylabel("Vérités terrain")
    plt.tight_layout()
    cm_path = f"{FIG_DIR}/xgb_confusion_matrix.png"
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)
    plt.close()

    # --- Courbe ROC
    RocCurveDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("ROC Curve - XGBoost")
    plt.tight_layout()
    roc_path = f"{FIG_DIR}/xgb_roc_curve.png"
    plt.savefig(roc_path)
    mlflow.log_artifact(roc_path)
    plt.close()

    # --- Courbe Precision-Recall
    PrecisionRecallDisplay.from_estimator(model, X_test_processed, y_test)
    plt.title("Precision-Recall Curve - XGBoost")
    plt.tight_layout()
    pr_path = f"{FIG_DIR}/xgb_pr_curve.png"
    plt.savefig(pr_path)
    mlflow.log_artifact(pr_path)
    plt.close()

    # --- Importance des features
    importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(
        ascending=False
    )
    plt.figure(figsize=(8, 5))
    imp_df = pd.DataFrame({'importance': importances[:10].values, 'feature': importances[:10].index})
    sns.barplot(data=imp_df, x='importance', y='feature', palette="rocket", hue='feature', legend=False)
    plt.title("Top 10 Feature Importances - XGBoost")
    plt.tight_layout()
    feat_imp_path = f"{FIG_DIR}/xgb_feature_importances.png"
    plt.savefig(feat_imp_path)
    mlflow.log_artifact(feat_imp_path)
    plt.close()

    print("[bold cyan]✅ XGBoost logué dans MLflow avec succès[/bold cyan]")

# 🔎 Résumé local
print(
    f"""
📊 Résumé des métriques - XGBoost :
  Train Accuracy: {final_metrics_test['train_accuracy']:.3f}
  Test Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  Overfitting Gap (Accuracy): {final_metrics_test['overfit_gap_accuracy']:.3f}
  ---
  Train F1 classe 0: {final_metrics_test['train_f1_0']:.3f}
  Test F1 classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Overfitting Gap (F1_0): {final_metrics_test['overfit_gap_f1_0']:.3f}
  ---
  Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  ROC-AUC:   {final_metrics_test['final_test_auc']:.3f}
  Precision classe 0: {final_metrics_test['final_test_precision_0']:.3f}
  Recall classe 0:    {final_metrics_test['final_test_recall_0']:.3f}
  F1-score classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Precision classe 1: {final_metrics_test['final_test_precision_1']:.3f}
  Recall classe 1:    {final_metrics_test['final_test_recall_1']:.3f}
  F1-score classe 1:  {final_metrics_test['final_test_f1_1']:.3f}
  FPR:       {fp_rate:.3f}
  FNR:       {fn_rate:.3f}
"""
)

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/xgboost/sklearn.py:1115: UserWarning: [15:30:07] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575

✅ XGBoost logué dans MLflow avec succès

📊 Résumé des métriques - XGBoost :
  Train Accuracy: 0.722
  Test Accuracy:  0.699
  Overfitting Gap (Accuracy): 0.023
  ---
  Train F1 classe 0: 0.594
  Test F1 classe 0:  0.546
  Overfitting Gap (F1_0): 0.048
  ---
  Accuracy:  0.699
  ROC-AUC:   0.788
  Precision classe 0: 0.424
  Recall classe 0:    0.767
  F1-score classe 0:  0.546
  Precision classe 1: 0.904
  Recall classe 1:    0.678
  F1-score classe 1:  0.775
  FPR:       0.233
  FNR:       0.322